In [ ]:
# aws SDK boto3 for python
%pip install boto3

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.42.37 which is incompatible.


In [8]:
import boto3
import pandas as pd


In [10]:
#List all S3 buckets
s3 = boto3.client("s3")
response = s3.list_buckets()

print("Existing buckets:")
for bucket in response['Buckets']:
    print(f'* {bucket["Name"]}')


Existing buckets:
* aws-de-labs-bucket


In [13]:
#Upload a file to S3 bucket
bucket_name = "aws-de-labs-bucket" #created a bucket befrorehand
s3.upload_file('../data/songs.csv', bucket_name, 'spotify_data/songs.csv')
# s3.upload_file('../data/users.csv', bucket_name, 'spotify_data/users.csv')
s3.upload_file('../data/streams1.csv', bucket_name, 'spotify_data/streams/streams1.csv')

In [ ]:
#Read the uploaded file from S3 bucket
obj = s3.get_object(Bucket= bucket_name, Key='spotify_data/songs.csv')
songs=pd.read_csv(obj['Body'])
songs.head(5)

,id,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [17]:
#List all files in S3 bucket that match the prefix """
response = s3.list_objects_v2(Bucket=bucket_name, Prefix='spotify_data/streams/')
files = [content['Key'] for content in response.get('Contents', []) if content['Key'].endswith('.csv')]
files

['spotify_data/streams/streams1.csv']

In [ ]:
#Delete a file from S3 bucket
s3.delete_object(Bucket=bucket_name, Key='spotify_data/songs.csv')
# s3.delete_object(Bucket=bucket_name, Key='spotify_data/users.csv')
s3.delete_object(Bucket=bucket_name, Key='spotify_data/streams/streams1.csv')

{'ResponseMetadata': {'RequestId': 'F12GV4P5QNJX5Q84',
  'HostId': '1XYRmXSpXsgmWgvGyUZkBSI2BWgiVxqwnqAb2iWr098XTCoEAy2IuZF24YMnjhtKf3zq0OIdIuI=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '1XYRmXSpXsgmWgvGyUZkBSI2BWgiVxqwnqAb2iWr098XTCoEAy2IuZF24YMnjhtKf3zq0OIdIuI=',
   'x-amz-request-id': 'F12GV4P5QNJX5Q84',
   'date': 'Thu, 29 Jan 2026 05:32:03 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [ ]:
#Delete a folder from S3 bucket
def delete_folder_from_s3(bucket_name, folder_prefix):
    s3_resource = boto3.resource('s3')
    bucket = s3_resource.Bucket(bucket_name)
    objects_to_delete = bucket.objects.filter(Prefix=folder_prefix)
    objects_to_delete.delete()
